# Narrative Lens Exploratory Data Analysis

In [89]:
import pandas as pd
import pymongo
import json
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

import matplotlib.pyplot as plt
import seaborn as sns


In [104]:
#mongo conection
try:
    client = MongoClient('mongodb://localhost:27017/')
    db = client['NarrativeLens']
    collection = db['tiktok_ads_us_toplikes']
    print("MongoDB connection successful")

except ConnectionFailure as e:
    print(f"Could not connect to MongoDB: {e}")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

MongoDB connection successful


In [105]:
# Load the mongodb collection into a pandas DataFrame
def load_mongo_collection_to_dataframe(collection):
    try:
        data = list(collection.find())
        if not data:
            print("No data found in the collection.")
            return pd.DataFrame()
        df = pd.DataFrame(data)
        # Drop the '_id' column if it exists
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
        return df
    except Exception as e:
        print(f"An error occurred while loading data from MongoDB: {e}")
        return pd.DataFrame()

In [106]:
# Load the data into a DataFrame
df = load_mongo_collection_to_dataframe(collection)


In [107]:
df.head()

,ad_title,brand_name,cost,ctr,favorite,id,industry_key,is_search,like,objective_key,tag,video_info,detail_analysis,industry,objective,scrap_datetime,video_name,creative_features
0,Great time killer!,Survival Game Master,2,0.01,False,7132878852058906625,label_23125000000,True,38213,campaign_objective_app_install,3.0,"{'vid': 'v0911dg40001cbta3ebc77u7vbp6b1gg', 'd...",This ad is using Product Review to catch audie...,"{'child': {'id': 23125000000, 'value': 'Games ...","{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_0_1,{'creative_theme': 'Promotional & Offer-Based'...
1,Oddly satisfying game,Gameworld Master,2,0.01,False,7109275920046178305,label_23125000000,True,14299,campaign_objective_app_install,3.0,"{'vid': 'v10033g50000caldhr3c77ub7mthrn5g', 'd...",This ad is using Strategy Focused to catch aud...,"{'child': {'id': 23125000000, 'value': 'Games ...","{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_1_1,"{'creative_theme': 'Product-Centric', 'creativ..."
2,My friend recommended me to play this game,Sugarcane Inc. Empire Tycoon,0,0.02,False,7077499601561305089,label_23125000000,True,2427,campaign_objective_app_install,NaN,"{'vid': 'v10033g50000cfgfgl3c77u9fehpnd8g', 'd...",,"{'child': {'id': 23125000000, 'value': 'Games ...","{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_2_1,"{'creative_theme': 'Educational & Explainer', ..."
3,download now,"Smart VPN - Fast, Stable",2,0.02,False,7176836203668783106,label_23125000000,True,124771,campaign_objective_app_install,3.0,"{'vid': 'v10033g50000cuqnqlnog65qepkj72vg', 'd...",This ad is using Oddly Satisfying to catch aud...,"{'child': {'id': 23125000000, 'value': 'Games ...","{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_3_1,"{'creative_theme': 'Lifestyle & Aspirational',..."
4,Play ten minutes a day to relieve stress!,Sugarcane Factory 3D,0,0.02,False,7077499507373899777,label_23125000000,True,2739,campaign_objective_app_install,NaN,"{'vid': 'v10033g50000cfl0uujc77u563dfii20', 'd...",,"{'child': {'id': 23125000000, 'value': 'Games ...","{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_4_1,"{'creative_theme': 'Product-Centric', 'creativ..."


In [108]:
#Flatening the json columns (video_info, indusrtry, objective, creative_fetures)
def flatten_json_columns(df):
    try:
        # Flatten the 'video_info' column
        video_info_df = pd.json_normalize(df['video_info'])
        video_info_df.columns = [f'video_info_{col}' for col in video_info_df.columns]
        
        # Flatten the 'industry' column
        industry_df = pd.json_normalize(df['industry'])
        industry_df.columns = [f'industry_{col}' for col in industry_df.columns]
        
        # Flatten the 'objective' column
        objective_df = pd.json_normalize(df['objective'])
        objective_df.columns = [f'objective_{col}' for col in objective_df.columns]
        
        # Flatten the 'creative_features' column
        creative_features_df = pd.json_normalize(df['creative_features'])
        creative_features_df.columns = [f'creative_features_{col}' for col in creative_features_df.columns]
        
        # Concatenate all DataFrames
        df_flattened = pd.concat([df.drop(columns=['video_info', 'industry', 'objective', 'creative_features']),
                                   video_info_df, industry_df, objective_df, creative_features_df], axis=1)
        
        return df_flattened
    except Exception as e:
        print(f"An error occurred while flattening JSON columns: {e}")
        return df


In [109]:
df = flatten_json_columns(df)

## Data Cleansing

In [110]:
df.drop(columns=['objective_id', 'industry_key', 'objective_key', 'industry_child.id', 'industry_parent.id', 'video_info_video_url.360p', 'video_info_video_url.480p', 'video_info_video_url.540p', 'video_info_video_url.720p', 'video_info_video_url.1080p'], inplace=True)

In [111]:
#Filling the missing values
print(df['tag'].value_counts())

#lets fill the missing values with 'unknown'
df.fillna({'tag':'unknown'}, inplace=True)

print(df['tag'].value_counts())

tag
2.0    126
3.0     95
Name: count, dtype: int64
tag
unknown    361
2.0        126
3.0         95
Name: count, dtype: int64


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 27 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ad_title                                      582 non-null    object 
 1   brand_name                                    582 non-null    object 
 2   cost                                          582 non-null    int64  
 3   ctr                                           582 non-null    float64
 4   favorite                                      582 non-null    bool   
 5   id                                            582 non-null    object 
 6   is_search                                     582 non-null    bool   
 7   like                                          582 non-null    int64  
 8   tag                                           582 non-null    object 
 9   detail_analysis                               582 non-null    obj

## EDA

In [119]:
# One of the potential questions that I want to answer is: Is UGC still a thing in TikTok ads (2024)?
# To do this, I will create a new column that indicates whether the ad is UGC or not
# I can tell if the ad is UGC or not by looking at the creative features

ugc_flags = {
  "creative_features_creative_concept": "User-generated content (UGC)",
  "creative_features_creative_concept": "Day-in-the-life story",
  "creative_features_creative_concept": "Before-and-after story",
  "creative_features_creative_concept": "Unboxing",
  "creative_features_creative_theme": "Testimonial & Social Proof",
  "creative_features_talent_type": "Influencers",
}

def is_ugc(row):
    for key, value in ugc_flags.items():
        if row[key] == value:
            return 1
    return 0

# Create a new column 'is_ugc' and apply the function
df['is_ugc'] = df.apply(is_ugc, axis=1)

In [120]:
df['is_ugc'].value_counts()

is_ugc
0    492
1     90
Name: count, dtype: int64

In [121]:
df['creative_features_creative_concept'].value_counts()

creative_features_creative_concept
Product demo                          253
None                                   98
Limited-time offer                     82
Day-in-the-life story                  36
Before-and-after story                 24
FAQ                                    14
Unboxing                               13
Expert review                           7
Behind-the-scenes                       7
Cinematic brand film                    7
Meme-based content                      7
Comparison                              6
Parody                                  5
Not Applicable                          4
Founder story                           4
Flash sale                              3
Event-driven                            3
Animated explainer                      3
User-generated content (UGC)            2
Aspirational creator collaboration      2
Member deal                             2
Name: count, dtype: int64

In [122]:
df['creative_features_format_production_style'].value_counts()

creative_features_format_production_style
Native Video                   532
Static Image                    25
Animation & Motion Graphics     19
High-Production Video            3
Carousel                         2
Gamified Experience              1
Name: count, dtype: int64

In [123]:
df['creative_features_creative_theme'].value_counts()

creative_features_creative_theme
Product-Centric                 221
Promotional & Offer-Based       129
Educational & Explainer          58
Lifestyle & Aspirational         49
Testimonial & Social Proof       37
Humor & Entertainment            32
Not Applicable                   24
Brand Story & Mission-Driven     20
Trend-Based & Reactive           12
Name: count, dtype: int64

In [124]:
df['creative_features_talent_type'].value_counts()

creative_features_talent_type
None                                        229
Customers                                   115
Unclear                                     101
Influencers                                  50
Actors                                       44
Combination of actors and customers          25
Experts                                      16
Combination of influencers and customers      2
Name: count, dtype: int64

In [127]:
#Videos that I think are UGC
df.loc[df['video_name'] == 'ad_11000000000_3_18_1']

,ad_title,brand_name,cost,ctr,favorite,id,is_search,like,tag,detail_analysis,...,industry_parent.value,objective_value,creative_features_creative_theme,creative_features_creative_concept,creative_features_format_production_style,creative_features_talent_type,creative_features_demographic_representation,creative_features_audience_focus,creative_features_campaign_objective,is_ugc
173,The best mobile phone holder for car navigation,Anthelper,2,0.01,False,7209659939894804481,True,9611,unknown,This ad is using Product Demonstration to catc...,...,Vehicle & Transportation,Conversions,Product-Centric,Product demo,Native Video,None,Unclear,Problem Aware,Sales,0


In [57]:
from ydata_profiling import ProfileReport

/Users/camilojaureguiberry/Documents/Projects/Developments/NarrativeLens/NarrativeLens/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
report = ProfileReport(df, title="Quick EDA Report", explorative=True)
report.to_file("report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 56.90it/s]


In [64]:
#Export the DataFrame to a CSV file
df.to_csv('../../data/datasets/tiktok_ads_us_toplikes.csv', index=False)


